In [1]:
import numpy as np
import matplotlib.pyplot as plt
import os
import cv2
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

# confirm TensorFlow sees the GPU
from tensorflow.python.client import device_lib
assert 'GPU' in str(device_lib.list_local_devices())
from tensorflow import GPUOptions, Session, ConfigProto

# confirm Keras sees the GPU
from keras import backend
assert len(backend.tensorflow_backend._get_available_gpus()) > 0

gpu_options = GPUOptions(per_process_gpu_memory_fraction = 0.8)           #Assign portion of memory for training this model
sess = Session(config = ConfigProto(gpu_options = gpu_options))


#Uncomment to run on CPU
#os.environ['CUDA_VISIBLE_DEVICES'] = '-1'

Using TensorFlow backend.
W0902 15:07:40.764135  7492 deprecation_wrapper.py:119] From c:\users\shinigami shrek\appdata\local\programs\python\python36\lib\site-packages\keras\backend\tensorflow_backend.py:174: The name tf.get_default_session is deprecated. Please use tf.compat.v1.get_default_session instead.

W0902 15:07:40.766162  7492 deprecation_wrapper.py:119] From c:\users\shinigami shrek\appdata\local\programs\python\python36\lib\site-packages\keras\backend\tensorflow_backend.py:181: The name tf.ConfigProto is deprecated. Please use tf.compat.v1.ConfigProto instead.

W0902 15:07:40.767127  7492 deprecation_wrapper.py:119] From c:\users\shinigami shrek\appdata\local\programs\python\python36\lib\site-packages\keras\backend\tensorflow_backend.py:186: The name tf.Session is deprecated. Please use tf.compat.v1.Session instead.

W0902 15:07:40.819984  7492 deprecation_wrapper.py:119] From c:\users\shinigami shrek\appdata\local\programs\python\python36\lib\site-packages\keras\backend\te

In [10]:
from tensorflow.keras.callbacks import TensorBoard
import datetime
NAME = "Galaxy-zoo-{:%Y-%m-%d %H:%M:%S}".format(datetime.datetime.now())

tensorboard = TensorBoard(log_dir = 'logs\{}'.format(NAME))

DIR = "E:\Galaxy Zoo\Data\images_training_rev1"

train_path = DIR + "\Train"
validate_path = DIR + "\Validate"

df = pd.read_csv("training_solutions_rev1/training_solutions_rev1.csv")

df.set_index('GalaxyID', inplace = True)

In [3]:
def img_processor(path):
    img = cv2.imread(path)
    img = img[106:106*3,106:106*3,:]
    img = cv2.resize(img,(106,106), interpolation = cv2.INTER_CUBIC)
    return img

def Batch_generator(DIR):
    for img in os.listdir(DIR):
        im = img_processor(os.path.join(DIR,img))
        ind = int(img[:-4])
        y_train = df.loc[ind].values
        X_train = np.array(im)
        X_train = X_train.reshape(1,106,106,3)
        y_train = y_train.reshape(1,37)
        yield(X_train,y_train)
        
def Validation_generator(DIR):
    for img in os.listdir(DIR):
        im = img_processor(os.path.join(DIR,img))
        ind = int(img[:-4])
        y_valid = df.loc[ind].values
        X_valid = np.array(im)
        X_valid = X_valid.reshape(1,106,106,3)
        y_valid = y_valid.reshape(1,37)
        yield(X_valid,y_valid)

In [4]:
from keras.models import Sequential
from keras.layers import Dense,Dropout,Activation,Flatten,Conv2D,MaxPooling2D,ZeroPadding2D,Lambda
from keras.optimizers import SGD, RMSprop, Adam

In [5]:
def Conv(model,layers,filters):
    for i in range(layers):
        model.add(ZeroPadding2D((1,1))) 
        model.add(Conv2D((filters), kernel_size = (3,3), activation = 'relu'))
        
    model.add(MaxPooling2D(pool_size=(2,2), strides = (2,2)))
    
def FC_block(model,filters):
    model.add(Dense(filters, activation='relu'))
    model.add(Dropout(0.5))
    
def VGG16():
    model = Sequential()
    model.add(Lambda(lambda x : x, input_shape=(106,106,3)))

    Conv(model,2,64)
    Conv(model,2,128)
    Conv(model,3,256)
    Conv(model,3,512)
    Conv(model,3,512)
    
    model.add(Flatten())
    FC_block(model,4096)
    FC_block(model,4096)
    FC_block(model,1000)
    
    model.add(Dense(37))
    model.add(Activation("sigmoid"))

    return model

In [6]:

model = VGG16()

model.compile(optimizer = RMSprop(lr=1e-6), loss='mean_squared_error', metrics = ['accuracy'])


W0902 15:07:43.943867  7492 deprecation_wrapper.py:119] From c:\users\shinigami shrek\appdata\local\programs\python\python36\lib\site-packages\keras\backend\tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0902 15:07:43.971828  7492 deprecation_wrapper.py:119] From c:\users\shinigami shrek\appdata\local\programs\python\python36\lib\site-packages\keras\backend\tensorflow_backend.py:3976: The name tf.nn.max_pool is deprecated. Please use tf.nn.max_pool2d instead.

W0902 15:07:44.103472  7492 deprecation.py:506] From c:\users\shinigami shrek\appdata\local\programs\python\python36\lib\site-packages\keras\backend\tensorflow_backend.py:3445: calling dropout (from tensorflow.python.ops.nn_ops) with keep_prob is deprecated and will be removed in a future version.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
W0902 15:07:44.172286  7492 deprecation_wrap

In [7]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lambda_1 (Lambda)            (None, 106, 106, 3)       0         
_________________________________________________________________
zero_padding2d_1 (ZeroPaddin (None, 108, 108, 3)       0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 106, 106, 64)      1792      
_________________________________________________________________
zero_padding2d_2 (ZeroPaddin (None, 108, 108, 64)      0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 106, 106, 64)      36928     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 53, 53, 64)        0         
_________________________________________________________________
zero_padding2d_3 (ZeroPaddin (None, 55, 55, 64)        0         
__________

In [8]:
batch_size = 32

steps_per_batch = int(len(os.listdir(train_path))/batch_size)
val_steps_per_batch = int(len(os.listdir(validate_path))/batch_size)

In [11]:
history = model.fit_generator(
                    Batch_generator(train_path),
                    samples_per_epoch=steps_per_batch,
                    epochs = 5,
                    validation_data=Validation_generator(validate_path),
                    nb_val_samples=val_steps_per_batch,
                    verbose = 1,
                    callbacks = [tensorboard]
                   )

Epoch 1/5
1924/1924 [==============================] - 147s 76ms/step - loss: 0.0830 - acc: 0.3701 - val_loss: 0.0327 - val_acc: 0.5825
Epoch 2/5
1924/1924 [==============================] - 147s 77ms/step - loss: 0.0460 - acc: 0.4891 - val_loss: 0.0300 - val_acc: 0.5908
Epoch 3/5
1924/1924 [==============================] - 147s 77ms/step - loss: 0.0408 - acc: 0.5130 - val_loss: 0.0290 - val_acc: 0.5678
Epoch 4/5
1924/1924 [==============================] - 148s 77ms/step - loss: 0.0375 - acc: 0.5229 - val_loss: 0.0268 - val_acc: 0.6033
Epoch 5/5
1924/1924 [==============================] - 147s 76ms/step - loss: 0.0358 - acc: 0.5172 - val_loss: 0.0272 - val_acc: 0.5992
